In [1]:

import os
# import numpy as np
import tensorflow as tf
# import glob
# import geopandas as gpd
import json
# import configparser 
# import rioxarray as rioxr
# import xarray as xr
# import pathlib
# import pandas as pd
# import xarray as xr
import sys

## EE-ify the model
from tensorflow.python.tools import saved_model_utils


In [4]:
homedir = '/Users/tud500158/Library/Mobile Documents/com~apple~CloudDocs/Documents/Documents - TUD500158/github/AutomatedDamageDetection/'
encoder_dir = os.path.join(homedir,'training/2023-05/model_1684233861_L2_w20_k5_f16_a20/encoder_epoch_9')


''' ----------
Define model to load
------------'''

if encoder_dir is None:
    raise NameError('No encoder specified. Run script as "python this_script.py /path/to/model_dir/encoder_dir"')

if os.path.isdir(encoder_dir):
    path_to_encoder_epoch = encoder_dir
else:
    path_to_encoder_epoch = os.path.join(os.path.expanduser('~'), encoder_dir)

## Retrieve master directory and its name
path_to_model = os.path.split(path_to_encoder_epoch)[0]
model_dir = os.path.basename(path_to_model)


In [71]:


''' ----------
Load model/encoder
------------'''

epoch_num = path_to_encoder_epoch.split('_')[-1]
encoder = tf.keras.models.load_model(path_to_encoder_epoch,compile=False) # compile=True does not work


# Get latent_dim (of sampling layer)
latent_dim = encoder.layers[-1].output_shape[-1] 

print('----\n loaded encoder {}'.format(path_to_encoder_epoch) )
print('LDIM:',latent_dim)

----
 loaded encoder /Users/tud500158/Library/Mobile Documents/com~apple~CloudDocs/Documents/Documents - TUD500158/github/AutomatedDamageDetection/training/2023-05/model_1684233861_L2_w20_k5_f16_a20/encoder_epoch_9
LDIM: 2


# EEification

EEIfication prepares the model for hosting on [Google AI Platform](https://cloud.google.com/ai-platform).  Learn more about EEification from [this doc](https://developers.google.com/earth-engine/tensorflow#interacting-with-models-hosted-on-ai-platform).  First, get (and SET) input and output names of the nodes.  **CHANGE THE OUTPUT NAME TO SOMETHING THAT MAKES SENSE FOR YOUR MODEL!**  Keep the input name of 'array', which is how you'll pass data into the model (as an array image).

In [22]:


''' ----------
EE-ify the model:
Set input and output names so earth engine can read it
from: https://mygeoblog.com/2020/09/22/rice-mapping-using-deep-neural-networks/
and https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_TensorFlow_AI_Platform.ipynb#scrollTo=w49O7n5oTS4w
------------'''

MODEL_DIR = encoder_dir
meta_graph_def = saved_model_utils.get_meta_graph_def(MODEL_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None

# print(inputs.items())
for k,v in inputs.items():
    # print(k,v)
    input_name = v.name
    break

output_name = None
for k,v in outputs.items():
    output_name = v.name
    break
    
# print(input_name)
# print(output_name)

# Make a dictionary that maps Earth Engine outputs and inputs to 
# AI Platform inputs and outputs, respectively.
label='ldim'
input_dict = "'" + json.dumps({input_name: "array"}) + "'" # keep "array" as input name
output_dict = "'" + json.dumps({output_name: label}) + "'"

print(input_dict)
print(output_dict)



'{"serving_default_input_1:0": "array"}'
'{"StatefulPartitionedCall:1": "damage"}'


## Run the EEifier

The actual EEification is handled by the `earthengine model prepare` command.  Note that you will need to set your Cloud Project prior to running the command

In [35]:

# name = "mizeboud"
OUTPUT_BUCKET = 'ee-data_export/test'
PROJECT = 'ee-iceshelf-gee4geo' # 'servirtensorflow'

# Put the EEified model next to the trained model directory.
EEIFIED_DIR = 'gs://' + OUTPUT_BUCKET + '/eeified_pixel_model'

# # You need to set the project before using the model prepare command.
# !earthengine set_project {PROJECT}
# !earthengine model prepare --source_dir {MODEL_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

# You need to set the project before using the model prepare command.
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir "{MODEL_DIR}" --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

Successfully saved project id
Success: model at 'gs://ee-data_export/test/eeified_pixel_model' is ready to be hosted in AI Platform.


# Deploy and host the EEified model on AI Platform

Now there is another TensorFlow `SavedModel` stored in `EEIFIED_DIR` ready for hosting by AI Platform.  Do that from the `gcloud` command line tool, installed in the Colab runtime by default.  Be sure to specify a regional model with the `REGION` parameter.  Note that the `MODEL_NAME` must be unique.  If you already have a model by that name, either name a new model or a new version of the old model.  The [Cloud Console AI Platform models page](https://console.cloud.google.com/ai-platform/models) is useful for monitoring your models.

**If you change anything about the trained model, you'll need to re-EEify it and create a new version!**

In [36]:
EEIFIED_DIR

'gs://ee-data_export/test/eeified_pixel_model'

In [44]:
MODEL_NAME = 'VAE_model'
VERSION_NAME = 'v0'
REGION = 'europe-west4' # billing region

!gcloud ai-platform models create {MODEL_NAME} \
  --project {PROJECT} \
  --region {REGION}

!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --region {REGION} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --framework "TENSORFLOW" \
  --runtime-version=2.3 \
  --python-version=3.10

Using endpoint [https://europe-west4-ml.googleapis.com/]
Created ai platform model [projects/ee-iceshelf-gee4geo/models/pixel_demo_model_maaike].
Using endpoint [https://europe-west4-ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


# Connect to the hosted model from Earth Engine

1. Generate the input imagery.  This should be done in exactly the same way as the training data were generated.  See [this example notebook](http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb) for details.
2. Connect to the hosted model.
3. Use the model to make predictions.
4. Display the results.

Note that it takes the model a couple minutes to spin up and make predictions.

In [40]:
import ee
# ee.Authenticate()
ee.Initialize()

In [43]:
import folium
import tensorflow as tf

In [ ]:

# Point to the model hosted on AI Platform.  If you specified a region other
# than the default (us-central1) at model creation, specify it here.
model = ee.Model.fromAiPlatformPredictor(
    projectName=PROJECT,
    modelName=MODEL_NAME,
    version=VERSION_NAME,
    region=REGION,
    # Can be anything, but don't make it too big.
    inputTileSize=[8, 8],
    # Keep this the same as your training data.
    proj=ee.Projection('EPSG:4326').atScale(30),
    fixInputProj=True,
    # Note the names here need to match what you specified in the
    # output dictionary you passed to the EEifier.
    outputBands={'output': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    },
)



In [62]:

# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(BANDS).divide(10000)

def getQABits(image, start, end, newName):
    # Compute the bits we need to extract.
    pattern = 0;
    for i in range(start,end+1):
    # for (var i = start; i <= end; i++) {
       pattern += i**2 # Math.pow(2, i);
    # Return a single band image of the extracted QA bits, giving the band
    # a new name.
    return image.select([0], [newName]) .bitwiseAnd(pattern).rightShift(start);

# Function to mask clouds in Sentinel2
def S2maskClouds(dummy):
  # S2  = require('users/earthmapps/public:methods/S2.js')
  # Select the QA band containing the cloud mask values
  QA = dummy.select('QA60');
  # Get the internal_cloud_algorithm_flag bit.
  internalCloud = getQABits(QA, 10, 11, 'CloudQA');
  # Return an image masking out cloudy areas (internal cloud gt 0)
  return dummy.updateMask(internalCloud.eq(0));


# Use Landsat 8 surface reflectance data for predictors.
L8SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
# Use these bands for prediction.
BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
BANDS = ['B4','B3','B2']
LABEL='damage' 


# // Define start & end dates
start = '2019-11-1'
end = '2020-3-1'

# iceShelves = ee.FeatureCollection('users/izeboudmaaike/ne_10m_antarctic_ice_shelves_polys')
gridTiles = ee.FeatureCollection('projects/ee-izeboudmaaike/assets/gridTiles_iceShelves')

# // Load collection & apply standard metadata filtes
S2col = (ee.ImageCollection("COPERNICUS/S2_SR") 
            # // .filterBounds(gridTiles_iShlf.first().geometry()) // Filter by a single tile
            .filterBounds(gridTiles)  # // filter by tile collection
            .filterDate(start,end)  # // Filter by date
            .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',30)  # // Filter by metadata to get most cloudy scenes out
            .sort('CLOUDY_PIXEL_PERCENTAGE',True)
 ) #// Sort by cloudy pixel percentage to get the least cloudy images first
S2col = S2col.map(S2maskClouds)
            

# These names are used to specify properties in the export of 
# training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)

# List of fixed-length features, all of which are float32.
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES
]

# Dictionary with feature names as keys, fixed-length features as values.
FEATURES_DICT = dict(zip(FEATURE_NAMES, columns))

In [84]:
# import ee 



# The image input data is a 2018 cloud-masked median composite.
# image = L8SR.filterDate('2018-01-01', '2018-12-31').map(maskL8sr).median()
image = S2col.reduce(ee.Reducer.firstNonNull())

# Get a map ID for display in folium.
rgb_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3, 'format': 'png'}
rgb_vis = {'bands': ['B4_first', 'B3_first', 'B2_first'], 'min': 4e3, 'max': 12e3, 'format': 'png'}
mapid = image.getMapId(rgb_vis)

# Turn into an array image for input to the model.
array_image = image.float().toArray()

array_image

In [69]:
REGION

'europe-west4'

In [64]:

# model.predictImage outputs a one dimensional array image that
# packs the output nodes of your model into an array.  These
# are class probabilities that you need to unpack into a 
# multiband image with arrayFlatten().  If you want class
# labels, use arrayArgmax() as follows.
predictions = model.predictImage(array_image)
probabilities = predictions.arrayFlatten([['bare', 'veg', 'water']]) # 3 classes
label = predictions.arrayArgmax().arrayGet([0]).rename('label')

# print(predictions)

In [80]:
print(predictions.getInfo())
predictions.select('output').getInfo()

{'type': 'Image', 'bands': [{'id': 'output', 'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 1}, 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, 0, 0, 0.00026949458523585647, 0]}]}


{'type': 'Image',
 'bands': [{'id': 'output',
   'data_type': {'type': 'PixelType', 'precision': 'float', 'dimensions': 1},
   'crs': 'EPSG:4326',
   'crs_transform': [0.00026949458523585647,
    0,
    0,
    0,
    0.00026949458523585647,
    0]}]}

In [85]:

# Get map IDs for display in folium.
probability_vis = {
    'bands': ['bare', 'veg', 'water'], 'max': 0.5, 'format': 'png'
}
label_vis = {
    'palette': ['red', 'green', 'blue'], 'min': 0, 'max': 2, 'format': 'png'
}
probability_mapid = probabilities.getMapId(probability_vis)
label_mapid = label.getMapId(label_vis)

# Visualize the input imagery and the predictions.
# map = folium.Map(location=[37.6413, -122.2582], zoom_start=11)
map = folium.Map(location=[-75, 5], zoom_start=4)

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='composite',
  ).add_to(map)
folium.TileLayer(
  tiles=label_mapid['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='predicted label',
).add_to(map)
folium.TileLayer(
  tiles=probability_mapid['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='probability',
).add_to(map)
map.add_child(folium.LayerControl())
map